# Credit Card Spending Analysis Project 2017 For Display

In [2]:
from IPython.display import HTML# Hide code
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
#import plotly.plotly as py
#import plotly.graph_objs as go
#import plotly.tools as tls
#tls.set_credentials_file(username='drozen', api_key='GTP8SX2KBqr3loYdTVb6')

In [ ]:
df = pd.read_csv('data/CC2017ForDisplay.csv')

In [ ]:
df = df[df['Transaction']=='DEBIT'] 
df = df[df['Memo'].notnull()] # remove null entries

In [ ]:
# credit card payments
dfCr = pd.read_csv('data/CC2017ForDisplay.csv')
dfCr = dfCr[dfCr['Transaction']=='CREDIT'] 
dfCR= dfCr[dfCr['Memo'].notnull()] # remove null entries

### Total Amount of Returns

In [ ]:
dfCr[dfCr['Name']!='PREAUTHORIZED PAYMENT'].sum()[4]  # amount of returns

In [ ]:
dfPay = dfCr[dfCr['Name']=='PREAUTHORIZED PAYMENT']

In [ ]:
df['Amount'] = df['Amount'].apply(pd.to_numeric, errors='coerce') *-1 #convert to int and convert amounts to positive
df['Transaction date'] = pd.to_datetime(df['Transaction date']) # get date info
df['Month'] = df['Transaction date'].apply(lambda x : x.month)
df['Day'] = df['Transaction date'].apply(lambda x : x.day)
df['Day of Week'] = df['Transaction date'].apply(lambda x : x.dayofweek)


In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
DayOfWeek = list(map(lambda var: dmap[var],df['Day of Week']))
df['Day of Week'] = DayOfWeek

In [ ]:
### Transaction types

In [ ]:
def returnCategory(x):
    return x.split(' ~ Category: ')[1]

def returnName(x):
    return x.split(' ')[0]

In [ ]:
# create category column
df['Category']=df['Memo'].apply(returnCategory)

# create single name column
df['Single Name']=df['Name'].apply(returnName)

## Transaction Counts

In [ ]:
#df['Name'].value_counts().head(10)

## Find total spending category (eg. gas) purchase amounts for the year

In [ ]:
df['Memo'].apply(returnCategory).value_counts()

In [ ]:
dfCat = df.groupby(['Category','Day of Week']).sum()
dfCatUnstacked = dfCat.unstack()['Amount']

In [ ]:
dfCatOnly = df.groupby(['Category']).sum()

In [ ]:
sns.set(font_scale=1.5)
fig, ax = plt.subplots()
fig.set_size_inches(18,6)
sns.barplot(ax=ax, x='Category', y = 'Amount', data = dfCatOnly.reset_index()
    ).set_title('Spending ($) per Transaction Category')
ax.set(ylabel='Amount Spent ($)')
#plt.tight_layout()

In [ ]:
sns.set(font_scale=1.5)

plt.figure(figsize=(12,6))
sns.heatmap(dfCatUnstacked).set_title('Heatmap Displaying $ Spent per Category per Day of Week')

In [ ]:
#sns.clustermap(dfCatUnstacked)

In [ ]:
sns.set(font_scale=1.5)
fig, ax = plt.subplots()
fig.set_size_inches(18,6)
sns.barplot(ax=ax, x='Day of Week', y = 'Amount', data = dfCat.reset_index(), hue = 'Category', estimator=sum
    ).set_title('Spending ($) per Transaction Category per Day of Week')
ax.set(ylabel='Amount Spent ($)')
#plt.tight_layout()

In [ ]:
sns.set(font_scale=1.5)
fig, ax = plt.subplots()
fig.set_size_inches(18,6)
sns.barplot(x='Category', y = 'Amount', data = dfCat.reset_index(), hue = 'Day of Week', estimator=sum, ax=ax).set_title('Spending ($) per Day of Week per Transaction Category')
ax.set(ylabel='Amount Spent ($)')

In [ ]:
categoryList = df['Memo'].apply(returnCategory).unique()

In [ ]:
nameAmtDict = {}
def nameAmt(name, df=df):
    dfCat = df[df['Name'].apply(returnName)==name]
    dfCat = dfCat.apply(pd.to_numeric, errors='coerce') #convert to int
    returnVal = round(dfCat['Amount'].sum()*1,2)
    #print("%.2f" % returnVal)
    return round(returnVal, 2)

In [ ]:
def updateDict(name):
    nameAmtDict[name] = nameAmt(name)

In [ ]:
#df['Name'].apply(returnName).value_counts().head(8)                                   

In [ ]:
# nameList = ["ABE'S", "PERSONAL", "NOFRILLS", "ESSO", "COSTCO", "WAL-MART", "WINNERS", "CARTER'S", "SAVOURS", "DR", "URTH", "HERMES"]
# nameAmtDict = {}
# for name in nameList:
#     updateDict(name)
# nameAmtDict

In [ ]:
# D = nameAmtDict

# fig, ax = plt.subplots(figsize=(15, 8))

# ax.bar(range(len(D)), D.values(), align='center')
# plt.xticks(range(len(D)), list(D.keys()))
# plt.title("Spending per Store")
# plt.xlabel("Store")
# plt.ylabel("Dollars $")
# plt.figure(figsize=(100,100))

# plt.show()

In [ ]:
# labels=list(D.keys())
# values=list(D.values())
# trace=go.Pie(labels=labels,values=values)
# #py.iplot([trace])

In [ ]:
categoryAmtDict = {}
def categoryAmt(category, df=df):
    dfCat = df[df['Memo'].apply(returnCategory)==category]
    dfCat = dfCat.apply(pd.to_numeric, errors='coerce') #convert to int
    returnVal = round(dfCat['Amount'].sum()*1,2)
    print("%.2f" % returnVal)
    return round(returnVal, 2)

In [ ]:
def updateDict(category):
    categoryAmtDict[category] = categoryAmt(category)

## Investigate sums by names

## Report sums only for at stores purchases more than 10 times

In [ ]:
threshold = 10

dfsum = df.groupby('Single Name').sum()[df.groupby('Single Name').count()['Name']>threshold]
#dfmean.reset_index().head(5) #turn index into a column

## Total Purchase Amount Per Store

In [ ]:
a4_dims = (30, 10)
sns.set(font_scale=2.3)

fig, ax = plt.subplots(figsize=a4_dims)
ax = sns.barplot(x='Single Name', y='Amount',data=dfsum.reset_index(),estimator=sum)
ax.set(xlabel='Store Name')

## Average Purchase Amount Per Store

In [ ]:
dfmean = df.groupby('Single Name').mean()[df.groupby('Single Name').count()['Name']>threshold]
a4_dims = (30, 10)
sns.set(font_scale=2.3)

fig, ax = plt.subplots(figsize=a4_dims)
ax = sns.barplot(x='Single Name', y='Amount',data=dfmean.reset_index(),estimator=sum)
ax.set(xlabel='Store Name')